In [126]:
import datetime
import requests
from bs4 import BeautifulSoup
import re

# 位置情報からその日の天気を返す
def get_weather_from_location(original_location):
    original_location = ''.join(original_location)
    # 住所の中から郵便番号を抽出する
    location = re.findall('\d{3}-\d{4}', original_location)
    # 1回目のスクレイピングでは住所を検索し、候補から取ってくる
    url = "https://weather.yahoo.co.jp/weather/search/?p=" + location[0]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    content = soup.find(class_="serch-table")
    # 2回目のスクレイピングで用いるURLを得る
    location_url = content.find('a').get('href')
    print(location_url)
    r = requests.get(location_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    content = soup.find(id='yjw_pinpoint_today').find_all('td')
    # print(content)
    warn = soup.find(id='wrnrpt').find_all('dd')
    # print(warn)
    info = []
    warn_info = []
    
    for each in content[1:]:
        info.append(each.get_text().strip('\n'))
    if(len(warn) != 0):
        for each in warn[0]:
            warn_info.append(each.get_text().strip('\n'))
    elif(len(warn) == 0):
        warn_info.append('None')
    warn_info  = '\n'.join(warn_info)
    # print(warn_info)
    # 時間
    time = info[:8]
    # 天気
    weather = info[9:17]
    # 気温
    temperature = info[18:26]
    # 湿度
    humidity = info[27:35]
    # 降水量
    precipitation = info[36:44]
    # 上の3つの情報を合わせる
    weather_info = [(time[i], weather[i], temperature[i],humidity[i],precipitation[i], warn_info) for i in range(8)]
    # print(weather_info)

    # result = [('{0[0]}: {0[1]}, {0[2]}°C'.format(weather_info[i])) for i in range(8)]
    result = [('{0[0]}: {0[1]}, {0[2]}°C, {0[3]}%, {0[4]}ml, {0[5]}'.format(weather_info[i])) for i in range(8)]
    result = ('{}の今日の天気は\n'.format(original_location) + '\n'.join(result) + 'です。\n')
    dt_now = datetime.datetime.now()
    result = result + '現在時刻は{}です。'.format(dt_now)

    return result

In [140]:
def get_weather(original_location,day):
    original_location = ''.join(original_location)
    # 住所の中から郵便番号を抽出する
    location = re.findall('\d{3}-\d{4}', original_location)
    # 1回目のスクレイピングでは住所を検索し、候補から取ってくる
    url = "https://weather.yahoo.co.jp/weather/search/?p=" + location[0]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    content = soup.find(class_="serch-table")
    # 2回目のスクレイピングで用いるURLを得る
    location_url = content.find('a').get('href')
    print(location_url)
    if (day == '1日'):
        result = get_tommorow_weather(location_url)
        result = ('{}の明日の天気は\n'.format(original_location) + '\n'.join(result) + 'です。')
        dt_now = datetime.datetime.now()
    elif (day == '1週間'):
        result1=get_tommorow_weather(location_url)
        result1= ('{}の明日の天気は\n'.format(original_location) + '\n'.join(result1)+ 'です。\n')
        result2= get_weekly_weather(location_url)
        result2= ('{}の1週間の天気は\n'.format(original_location) + '\n'.join(result2)+ 'です。')
        result = result1 + result2
        dt_now = datetime.datetime.now()
    elif(day == '今日'):
        result = get_today_weather(location_url)
        result = ('{}の今日の天気は\n'.format(original_location) + '\n'.join(result) + 'です。\n')
        dt_now = datetime.datetime.now()
        result = result + '現在時刻は{}です。'.format(dt_now)
    return result

In [141]:
def get_today_weather(location_url):
    r = requests.get(location_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    content = soup.find(id='yjw_pinpoint_today').find_all('td')
    # print(content)
    warn = soup.find(id='wrnrpt').find_all('dd')
    # print(warn)
    info = []
    warn_info = []
    
    for each in content[1:]:
        info.append(each.get_text().strip('\n'))
    if(len(warn) != 0):
        for each in warn[0]:
            warn_info.append(each.get_text().strip('\n'))
    elif(len(warn) == 0):
        warn_info.append('None')
    warn_info  = '\n'.join(warn_info)
    # print(warn_info)
    # 時間
    time = info[:8]
    # 天気
    weather = info[9:17]
    # 気温
    temperature = info[18:26]
    # 湿度
    humidity = info[27:35]
    # 降水量
    precipitation = info[36:44]
    # 上の3つの情報を合わせる
    weather_info = [(time[i], weather[i], temperature[i],humidity[i],precipitation[i], warn_info) for i in range(8)]
    # print(weather_info)

    # result = [('{0[0]}: {0[1]}, {0[2]}°C'.format(weather_info[i])) for i in range(8)]
    result = [('{0[0]}: {0[1]}, {0[2]}°C, {0[3]}%, {0[4]}ml, {0[5]}'.format(weather_info[i])) for i in range(8)]
    return result

In [142]:
def get_tommorow_weather(location_url):
    r = requests.get(location_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    content = soup.find(id='yjw_pinpoint_tomorrow').find_all('td')
    # print(content)
    info = []
    warn_info = []
    
    for each in content[1:]:
        info.append(each.get_text().strip('\n'))
    warn_info.append('None')
    warn_info  = '\n'.join(warn_info)
    # print(warn_info)
    # 時間
    time = info[:8]
    # 天気
    weather = info[9:17]
    # 気温
    temperature = info[18:26]
    # 湿度
    humidity = info[27:35]
    # 降水量
    precipitation = info[36:44]
    # 上の3つの情報を合わせる
    weather_info = [(time[i], weather[i], temperature[i],humidity[i],precipitation[i], warn_info) for i in range(8)]
    # print(weather_info)

    # result = [('{0[0]}: {0[1]}, {0[2]}°C'.format(weather_info[i])) for i in range(8)]
    result = [('{0[0]}: {0[1]}, {0[2]}°C, {0[3]}%, {0[4]}ml, {0[5]}'.format(weather_info[i])) for i in range(8)]
    return result

In [143]:
def get_weekly_weather(location_url):
    r = requests.get(location_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    content = soup.find(id='yjw_week').find_all('small')
    info = []
    for each in content[0:]:
        info.append(each.get_text().strip('\n'))
    weather_info = []
    for i in range(6):
        weather = info[i+1].replace('\n','').replace(' ','') + ': ' + info[i+8].strip() + ', ' + info[i+15].replace('\n','°C, ') + '°C, ' +info[i+22].strip() + '%'
        weather_info.append(weather)
    return weather_info

In [131]:
print(get_weather_from_location(['819-0383', '福岡県福岡市西区田尻１丁目１３−４']))

https://weather.yahoo.co.jp/weather/40/8210/40135.html
819-0383福岡県福岡市西区田尻１丁目１３−４の今日の天気は
0時: 曇り, 28°C, 78%, 0ml, 雷、強風、波浪
3時: 曇り, 28°C, 75%, 0ml, 雷、強風、波浪
6時: 曇り, 27°C, 75%, 0ml, 雷、強風、波浪
9時: 晴れ, 29°C, 69%, 0ml, 雷、強風、波浪
12時: 晴れ, 31°C, 66%, 0ml, 雷、強風、波浪
15時: 雨, 32°C, 61%, 1ml, 雷、強風、波浪
18時: 雨, 28°C, 83%, 0ml, 雷、強風、波浪
21時: 晴れ, 27°C, 82%, 0ml, 雷、強風、波浪です。
現在時刻は2022-09-13 19:19:22.751665です。


In [144]:
print(get_weather(['819-0383', '福岡県福岡市西区田尻１丁目１３−４'],'1週間'))

https://weather.yahoo.co.jp/weather/40/8210/40135.html
819-0383福岡県福岡市西区田尻１丁目１３−４の明日の天気は
0時: 晴れ, 27°C, 80%, 0ml, None
3時: 晴れ, 26°C, 79%, 0ml, None
6時: 晴れ, 26°C, 80%, 0ml, None
9時: 晴れ, 30°C, 69%, 0ml, None
12時: 晴れ, 33°C, 59%, 0ml, None
15時: 晴れ, 33°C, 58%, 0ml, None
18時: 晴れ, 30°C, 66%, 0ml, None
21時: 晴れ, 28°C, 73%, 0ml, Noneです。
819-0383福岡県福岡市西区田尻１丁目１３−４の1週間の天気は
9月16日(金): 晴れ, 34°C, 25°C, 10%
9月17日(土): 曇時々晴, 31°C, 25°C, 40%
9月18日(日): 曇一時雨, 32°C, 25°C, 60%
9月19日(月): 曇時々雨, 30°C, 24°C, 80%
9月20日(火): 曇一時雨, 28°C, 21°C, 60%
9月21日(水): 曇時々晴, 27°C, 20°C, 30%です。


In [150]:
dt_now = datetime.datetime.now()
print(str(dt_now).split(' ')[0])

2022-09-14
